# Cycloid visualization

*Also [ipywgt_tool.py](./ipywgt_tool.py) demo*

Interactive 3D visualization of a [cycloid](https://en.wikipedia.org/wiki/Cycloid) function by using [ipywidgets](https://pypi.org/project/ipywidgets/).


In [ ]:
import numpy as np
import ipywidgets
import plotly.graph_objects as go
import ipywgt_tool
import rlc_funcs

NUM_T_STEPS = 40
NUM_SAMPLES = 4
PM = np.array((1, -1))

fig = go.FigureWidget(layout=dict(
            # Suitable for 3D scatters
            height=600,
            margin={'b': 4, 'l': 4, 'r': 4, 't': 4},
            legend_x=0,
            scene = dict(
                    aspectratio=dict(x=1, y=2, z=1),
                    xaxis_title='imag (x)',
                    yaxis_title='time (y)',
                    zaxis_title='real (z)'),
            title_x=.5,
        ))
# Emphasise the axes
fig.update_layout(scene=dict(
        xaxis_zerolinecolor='gray',
        yaxis_zerolinecolor='gray',
        zaxis_zerolinecolor='gray',
    ))

# Summary annotation
info_annot = ipywgt_tool.add_annotation(fig, x=1, y=.9, xref='paper', yref='paper', showarrow=False)

# Main functions
fn_scatters = ipywgt_tool.add_multiple(fig, ipywgt_tool.add_scatter3d,
        name=('Euler\'s func', 'Euler\'s second', 'Euler\'s third'),
        mode='lines', line_dash='dash')

# Result (summed) function
fn_r_scatter = ipywgt_tool.add_scatter3d(fig, mode='lines', name=f'Result func')

# Sample values
sample_scatter = ipywgt_tool.add_scatter3d(fig, mode='markers', name=f'Sample data')
# At point0
sample0_scatters = ipywgt_tool.add_multiple(fig, ipywgt_tool.add_scatter3d,
        mode='lines',
        line_dash=('dot', 'dot', 'dash'),
        name=('Sample<sub>0</sub>', 'Sample<sub>0</sub> conj.', 'Sample<sub>0</sub> result'))
sample0_r_scatter = ipywgt_tool.add_scatter3d(fig, mode='lines', line_dash='dash', name=f'Sample<sub>0</sub> result')

def get_sample_vals(omegas, phis, trange, conjugated=False):
    """Reference Euler's function values"""
    vals = rlc_funcs.calc_euler_derivs(1, omegas, phis, trange)
    vals = vals[0]  # Take the only 0-th derivative
    if conjugated:
        vals_r = vals.mean(0)
    else:
        vals_r = vals.real
        vals = vals[np.newaxis, ...]
    return vals, vals_r

@ipywidgets.interact(
        a=(-10., 10.), b=(-2*np.pi, 4*np.pi),
        a0=(-5., 5.), b0=(-np.pi, np.pi),
        imag_b=True,
        add_conj=False,
        sample_dt=(1e-2, 2),
)
def update(a=-.5, b=np.pi/2, a0=0, b0=0, imag_b=True, add_conj=False, sample_dt=1/4):
    """Interactively update plot"""
    # Select trange [-<sample-period>, 3 <sample-periods>]
    trange = np.linspace(-NUM_SAMPLES*sample_dt, 3*NUM_SAMPLES*sample_dt, NUM_T_STEPS, endpoint=True)

    if imag_b:
        b = complex(0, b)
        b0 = complex(0, b0)
    if add_conj:
        b = b * PM
        b0 = b0 * PM
    omegas = np.asarray(a + b)
    phis = np.asarray(a0 + b0)

    annot_text = f'&#937; {np.round(omegas, 3)}, e<sup>&#937;</sup> {np.round(np.exp(omegas), 3)}'
    annot_text += f'<br>&#934; {np.round(phis, 3)}, e<sup>&#934;</sup> {np.round(np.exp(phis), 3)}<br>'
    if imag_b:
        annot_text += f'<br>Oscillation: {np.round(omegas.imag*180/np.pi,1)} deg/sec, {np.round(omegas.imag/2/np.pi, 2)} Hz' \
                f', {np.round(2*np.pi / (sample_dt * omegas.imag), 1)} samples/period'
    if (omegas.real != 0).any():
        if  (omegas.real <= 0).all():
            annot_text += f'<br>Attenuation: {np.round(1 / -omegas.real, 2)} sec'
        else:
            annot_text += f'<br>Intensification: {np.round(1 / omegas.real, 2)} sec'
    info_annot.text = annot_text

    # Function values
    fn, fn_r = get_sample_vals(omegas, phis, trange, add_conj)

    # Regular/conjugated Euler's functions along z,x, time along y
    ipywgt_tool.update_fig_objs_xyz(fn_scatters,
            zxy=ipywgt_tool.split_complex(fn, trange, broadcast=True))

    # Result function along z,x, time along y
    ipywgt_tool.update_fig_objs_xyz(fn_r_scatter,
            zxy=ipywgt_tool.split_complex(fn_r, trange, broadcast=True))

    # Sample data values
    strange = np.arange(NUM_SAMPLES) * sample_dt
    sfn, sfn_r = get_sample_vals(omegas, phis, strange, add_conj)
    ipywgt_tool.update_fig_objs_xyz(sample_scatter,
            zxy=ipywgt_tool.split_complex(sfn_r, strange, broadcast=True),
            text = [f'Sample<sub>{i}</sub>' for i, _ in enumerate(sfn_r)])
    # Sample0 values
    data = np.stack(([0]*sfn.shape[0], sfn[:, 0]), axis=-1)
    ipywgt_tool.update_fig_objs_xyz(sample0_scatters,
            zxy=ipywgt_tool.split_complex(data, strange[0], broadcast=True))

    ipywgt_tool.update_fig_objs_xyz(sample0_r_scatter,
            zxy=ipywgt_tool.split_complex([0, sfn_r[0]], strange[0], broadcast=True))

fig

## Projection as cycloid

Interpret various projections as a cycloid sum: [parallel](https://en.wikipedia.org/wiki/Parallel_projection),
[rectilinear](https://en.wikipedia.org/wiki/Gnomonic_projection) and [polar](https://en.wikipedia.org/wiki/Azimuthal_equidistant_projection).
Any projection can be represented as a sum of the source shape and its very specific reflection.

This is to visualize the shapes of such reflections.

In [ ]:
import rlc_reverse

fig = go.FigureWidget(layout=dict(
            margin={'b': 4, 'l': 4, 'r': 4, 't': 20}))
fig.update_xaxes(scaleanchor='y')

shape_scatt = ipywgt_tool.add_scatter(fig, name='Source shape')
projbase_scatt = ipywgt_tool.add_scatter(fig, mode='lines+markers', line_dash='dot', name='Projection base',
        marker_symbol=['circle-open', 'x'], marker_size=8)
proj_scatt = ipywgt_tool.add_scatter(fig, mode='lines', name='Projected shape', visible='legendonly')
ray_markers = ['circle-open', '3', 'x', '0', '1']       # Repeat in groups of 5: (4 ray-points + nan)
ray_scatt = ipywgt_tool.add_scatter(fig, mode='lines+markers', line_dash='dash', name='Projection rays',
        marker_symbol=ray_markers, marker_size=8, visible='legendonly')
reflect_scatt = ipywgt_tool.add_scatter(fig, name='Reflection', visible='legendonly')

# rlc_reverse decomposition results
decomposed_scatts = ipywgt_tool.add_multiple(fig, ipywgt_tool.add_scatter,
        name=('Decomposed A', 'Decomposed B'))

@ipywidgets.interact(
        proj_type={'Parallel': 0, 'Rectilinear': 1, 'Polar': 2},
        proj_base=(0., 5),
        shape_center=(1., 5),
        shape_scale=(.2, 2))
def update(proj_type, proj_base=3, shape_center=4, shape_scale=1):
    """Interactively update plot"""
    #
    # The shape is a circle
    #
    shape_phi = np.linspace(0, -.2+2j*np.pi, 20)
    shape_xy = np.exp(shape_phi) * shape_scale + shape_center
    ray_idx = np.argmax(shape_xy.imag) // 2     # Select some arbitray point

    # Shape
    shape_texts = [f'Point {i}: Phi={v:.3f} ({v.imag*180/np.pi:.1f} deg)' for i, v in enumerate(shape_phi)]
    ipywgt_tool.update_fig_objs_xyz(shape_scatt,
            xy=ipywgt_tool.split_complex(shape_xy),
            text=shape_texts)

    # Projection base
    ipywgt_tool.update_fig_objs_xyz(projbase_scatt,
            xy=ipywgt_tool.split_complex(np.array((0, proj_base))))

    # Projected shape
    proj_xy = shape_xy / proj_base
    if proj_type == 0:
        # Parallel projection
        proj_xy = proj_base * (1 + 1j * proj_xy.imag)
    elif proj_type == 1:
        # Rectilinear projection
        proj_xy = proj_base * (1 + 1j * proj_xy.imag / proj_xy.real)
    elif proj_type == 2:
        # Polar projection
        proj_xy = shape_xy * abs(proj_base) / abs(shape_xy)
    ipywgt_tool.update_fig_objs_xyz(proj_scatt,
            xy=ipywgt_tool.split_complex(proj_xy),
            text=shape_texts)

    # Reflection shape
    reflect_xy = 2*proj_xy - shape_xy
    ipywgt_tool.update_fig_objs_xyz(reflect_scatt,
            xy=ipywgt_tool.split_complex(reflect_xy),
            text=shape_texts)

    # Combine projection rays
    ray_xy = np.stack((
            # No center-point in parallel projection
            np.full_like(reflect_xy, 0 if proj_type else np.nan),
            reflect_xy, proj_xy, shape_xy), axis=-1)
    # Show 3 of them (around 'ray_idx')
    ray_xy = ray_xy[ray_idx-1:ray_idx+2]
    ipywgt_tool.update_fig_objs_xyz(ray_scatt,
            xy=ipywgt_tool.split_complex(ipywgt_tool.flatten_join(ray_xy, np.nan)),
            marker_symbol=ray_markers * ray_xy.shape[0])

    # rlc_reverse decomposition results
    src_adj = 0
    if True:
        # Decompose source shape
        decomp_xy = shape_xy
    else:
        # Decompose projected shape
        src_adj = -proj_base    # Suppress DC offset
        decomp_xy = proj_xy
    decomp_xy += src_adj
    _, decomp_xy = rlc_reverse.calc_exp_omega_phi(decomp_xy, conjugated=True)
    decomp_xy -= src_adj
    ipywgt_tool.update_fig_objs_xyz(decomposed_scatts,
            xy=ipywgt_tool.split_complex(decomp_xy))

fig